In [1]:
import sys
sys.path.append("../")
import yaml
import logging
import requests
from pprint import pprint
import json
import time
import os
from pymongo import MongoClient
import dateparser
import psutil
from nlp_utils import NLPUtils
from lda_module import LdaModule
import pandas as pd
from pprint import pprint

In [2]:
lda_module = LdaModule(lang='en',trained=True)

In [3]:
text = """The Centers for Disease Control and Prevention (CDC) said the novel coronavirus, or COVID-19, is spread mainly from person-to-person by those in close contact, or through coughing and sneezing by someone who’s infected.

Symptoms of the coronavirus can show up between two and 14 days of exposure, health officials say. Symptoms include fever, cough, and shortness of breath.

For most people, COVID-19 causes only mild or moderate symptoms, such as fever and cough. But some severe cases can lead to death.

Most people can recover from the virus at home using over-the-counter medications to treat their symptoms.

Some people who have the virus don’t show any symptoms, but they can still spread it to others. The CDC estimates that up to 35% of all cases are asymptomatic."""

In [4]:
nlp_utils = NLPUtils(lang='en')

In [5]:
parsed_text = nlp_utils.parse_text(text)

In [6]:
topics = lda_module.model.show_topics(formatted=False, num_topics=20, num_words=5)

In [7]:
topics

[(0,
  [('spread', 0.011918477),
   ('airport', 0.011631478),
   ('new_coronavirus', 0.009209555),
   ('patient', 0.009034123),
   ('official', 0.008796643)]),
 (1,
  [('new', 0.0132563915),
   ('city', 0.011033031),
   ('infect', 0.008027595),
   ('find', 0.0069969166),
   ('traveler', 0.0067912703)]),
 (2,
  [('Wuhan_Municipal_Health_Commission', 0.009482136),
   ('death_new_type', 0.00768391),
   ('people_critical_condition', 0.007521409),
   ('total_41_suffer_pneumonia', 0.0073302193),
   ('cause_preliminarily_determined_new', 0.0072804047)]),
 (3,
  [('spread', 0.015246936),
   ('disease', 0.011597013),
   ('country', 0.008807385),
   ('new', 0.008277978),
   ('airport', 0.0077647427)]),
 (4,
  [('new', 0.010447913),
   ('city', 0.010402071),
   ('spread', 0.009488106),
   ('new_coronavirus', 0.007017712),
   ('authority', 0.006671207)]),
 (5,
  [('spread', 0.008978136),
   ('disease', 0.008506834),
   ('identify', 0.0070273457),
   ('new', 0.0069914353),
   ('World_Health_Organiz

In [8]:
topics[lda_module.model[lda_module.dictionary.doc2bow(parsed_text)][0][0]][1]

[('spread', 0.011918477),
 ('airport', 0.011631478),
 ('new_coronavirus', 0.009209555),
 ('patient', 0.009034123),
 ('official', 0.008796643)]

In [9]:
document_topic_info = {}
for el in lda_module.model[lda_module.dictionary.doc2bow(parsed_text)]:
    document_topic_info[el[0]] = [round(el[1], 2), topics[el[0]][1]]

In [10]:
document_topic_info

{0: [0.16,
  [('spread', 0.011918477),
   ('airport', 0.011631478),
   ('new_coronavirus', 0.009209555),
   ('patient', 0.009034123),
   ('official', 0.008796643)]],
 3: [0.37,
  [('spread', 0.015246936),
   ('disease', 0.011597013),
   ('country', 0.008807385),
   ('new', 0.008277978),
   ('airport', 0.0077647427)]],
 12: [0.38,
  [('new', 0.009447029),
   ('test', 0.007193104),
   ('spread', 0.006554796),
   ('patient', 0.0065354817),
   ('week', 0.0064059366)]],
 15: [0.04,
  [('new', 0.010423159),
   ('spread', 0.010064991),
   ('travel', 0.009329155),
   ('official', 0.008044628),
   ('country', 0.007861818)]],
 18: [0.04,
  [('travel', 0.009961029),
   ('new', 0.009395931),
   ('know', 0.008163111),
   ('country', 0.007936068),
   ('spread', 0.0071191764)]]}

In [ ]:
document_info = pd.DataFrame([(el[0], round(el[1], 2), topics[el[0]][1])
                                      for el in lda_module.model[lda_module.dictionary.doc2bow(parsed_text)]],
                                     columns=['topic #', 'weight', 'words in topic'])

In [ ]:
document_info

In [ ]:
import spacy
sp = spacy.load('en_core_web_md')
all_stopwords = sp.Defaults.stop_words
def remove_mystopwords(sentence):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in tokens if not word in all_stopwords]
    return (" ").join(tokens_filtered)

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords

text = """The nationwide protests over the killing of George Floyd will test two competing notions of how voters respond to crises that provoke both anger and grief.

Could the demonstrations that have devolved into mayhem, looting and assault lead to victory for President Trump? Will they empower Republicans in the way that Richard Nixon used his “silent majority” and Ronald Reagan invoked the idea that “the jungle is always there, waiting to take us over” to propel themselves politically in the wake of the riots and rebellion in Watts, Newark and Detroit in the 1960s?

Or could Trump’s authoritarian reaction, both rhetorically (“You have to dominate,” “I am your president of law and order”) and actual (ordering military units and equipment to the nation’s capital) enrage and energize the knowledge class, white liberals and the young to more tightly ally with African-Americans, Hispanics and other minorities to produce a Democratic surge?

In Minnesota, where the protests first began, there are Democrats who worry that voters will turn to the right."""
filtered_sentence = remove_mystopwords(text)

print(filtered_sentence)